
# 🧠 Clasificación de Imágenes con CNN — CIFAR-10 (Colab)
**Autores:** Daniel Serrano y Alexander Arrosquipa  
**Universidad UNIE — Asignatura:** Lenguaje Natural y Compiladores  
**Fecha:** Octubre 2025

> Notebook listo para ejecutar en **Google Colab**. Contiene las fases 1–3: preparación del dataset, construcción del modelo CNN, entrenamiento, evaluación y visualización de resultados, además de la **Pregunta de Análisis**.


In [ ]:
# @title 🔧 Instalación (Colab)
# Esta celda asegura que las dependencias estén disponibles en Colab.
# (En Colab normalmente ya hay TensorFlow, pero lo dejamos por compatibilidad)
!pip -q install tensorflow matplotlib numpy


In [ ]:
# @title 📦 Importaciones
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Hacer que las figuras se vean bien en Colab
plt.rcParams['figure.figsize'] = (6,6)



## ⚙️ Fase 1 — Preparación del Dataset (CIFAR-10)

- **Carga** del dataset (`tensorflow.keras.datasets.cifar10`).
- **Normalización** de píxeles al rango \([0,1]\).
- **One-hot encoding** de etiquetas.
- **Visualización** de 9 imágenes con su clase.


In [ ]:
# @title 📥 Carga del dataset y preprocesamiento
# Carga
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalización al rango [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

# One-hot encoding
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# Resumen
print("Tamaño entrenamiento:", x_train.shape, "| Tamaño prueba:", x_test.shape)
print("Ejemplo etiqueta one-hot:", y_train[0])


In [ ]:
# @title 🖼️ Visualización de CIFAR-10 (9 ejemplos)
class_names = ['avión', 'automóvil', 'pájaro', 'gato', 'ciervo',
               'perro', 'rana', 'caballo', 'barco', 'camión']

fig, axes = plt.subplots(3, 3, figsize=(6,6))
for i, ax in enumerate(axes.flat):
    ax.imshow(x_train[i])
    ax.set_title(class_names[np.argmax(y_train[i])])
    ax.axis('off')
plt.tight_layout()
plt.show()



## 🧠 Fase 2 — Construcción del Modelo (CNN)

Arquitectura **Sequential** con dos bloques convolucionales (Conv2D + MaxPooling) y clasificador denso:
- `Conv2D(32, 3×3, ReLU)` → `MaxPooling(2×2)`  
- `Conv2D(64, 3×3, ReLU)` → `MaxPooling(2×2)`  
- `Flatten` → `Dense(64, ReLU)` → `Dense(10, Softmax)`


In [ ]:
# @title 🧩 Definición de la CNN
model = Sequential(name="CNN_CIFAR10_Base")
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3), name="Conv2D_1"))
model.add(MaxPooling2D(pool_size=(2,2), name="MaxPool_1"))
model.add(Conv2D(64, (3,3), activation='relu', name="Conv2D_2"))
model.add(MaxPooling2D(pool_size=(2,2), name="MaxPool_2"))
model.add(Flatten(name="Flatten"))
model.add(Dense(64, activation='relu', name="Dense_64"))
model.add(Dense(10, activation='softmax', name="Output"))

model.summary()



## 📈 Fase 3 — Entrenamiento y Evaluación

- **Compilación:** `optimizer=Adam`, `loss=categorical_crossentropy`, `metrics=['accuracy']`  
- **Entrenamiento:** 8 épocas, `batch_size=64`, `validation_split=0.1`  
- **Evaluación:** precisión y pérdida en el conjunto de **test**.


In [ ]:
# @title 🏋️ Compilación y entrenamiento (8 épocas)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    epochs=8,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

print("✅ Entrenamiento completado.")


In [ ]:
# @title 🧪 Evaluación en test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(f"📊 Resultado en test → Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f}")


In [ ]:
# @title 📊 Gráficas de precisión y pérdida
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12,5))

# Precisión
plt.subplot(1,2,1)
plt.plot(epochs, acc, 'o-', label='Entrenamiento')
plt.plot(epochs, val_acc, 'o-', label='Validación')
plt.title('Precisión')
plt.xlabel('Épocas'); plt.ylabel('Accuracy'); plt.legend()

# Pérdida
plt.subplot(1,2,2)
plt.plot(epochs, loss, 'o-', label='Entrenamiento')
plt.plot(epochs, val_loss, 'o-', label='Validación')
plt.title('Pérdida')
plt.xlabel('Épocas'); plt.ylabel('Loss'); plt.legend()

plt.tight_layout()
plt.show()



## 💬 Pregunta de Análisis

**¿Qué partes del modelo corresponden a (1) *Input*, (2) *Preprocessing*, (3) *Feature Extraction* y (4) *Classifier*?  
¿Cómo ha automatizado la CNN el proceso de *Feature Extraction*?**

**Respuesta (resumen técnico):**  
1) **Input**: los tensores de imágenes `x_train/x_test` con forma `(32,32,3)`.  
2) **Preprocessing**: la **normalización** de píxeles a \([0,1]\) y el **one-hot encoding** de etiquetas.  
3) **Feature Extraction**: las capas **Conv2D** y **MaxPooling2D**, que aprenden jerárquicamente filtros (bordes, texturas, formas) y reducen la dimensionalidad manteniendo información relevante.  
4) **Classifier**: `Flatten → Dense(64, ReLU) → Dense(10, Softmax)` que combinan las características extraídas y producen probabilidades por clase.  

La **CNN automatiza la extracción de características** porque los filtros de `Conv2D` **no están predefinidos**: se **aprenden** mediante **retropropagación del gradiente** para minimizar la **entropía cruzada**. Así, el modelo descubre representaciones discriminativas sin ingeniería manual de rasgos.
